## Trabalho 
#### Carregar o conjunto de dados do site de complaints e categorizar o tipo de produto baseado no texto do usuário

### 1. Carregando o dataset 

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('complaints.csv')

/srv/conda/envs/notebook/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
data.shape

(2124146, 18)

In [5]:
data = data[data['Consumer complaint narrative'].notna()]

In [6]:
X = data['Consumer complaint narrative'].head(10000)
y = data['Product'].head(10000)

In [7]:
X

5        Two accounts are still on my credit history af...
9        I opened a citi double cash card the beginning...
10       I filed 2 disputes with XXXX XXXX XXXX in XX/X...
11       In dispute of the loan # XXXX, for XXXX XXXX t...
31       Notice to whom it be of concern,It is YOU I ho...
                               ...                        
33159    Two rounds of dispute letters were sent to rem...
33161    Summit Financial is continuously reporting I o...
33162    I received a notification on XXXX XX/XX/2019 v...
33165    On XX/XX/XXXX I used a service call JPay to se...
33166    I am a victim of identity theft. The informati...
Name: Consumer complaint narrative, Length: 10000, dtype: object

In [8]:
y

5        Credit reporting, credit repair services, or o...
9                              Credit card or prepaid card
10       Credit reporting, credit repair services, or o...
11                                                Mortgage
31       Credit reporting, credit repair services, or o...
                               ...                        
33159    Credit reporting, credit repair services, or o...
33161    Credit reporting, credit repair services, or o...
33162                                      Debt collection
33165    Money transfer, virtual currency, or money ser...
33166    Credit reporting, credit repair services, or o...
Name: Product, Length: 10000, dtype: object

### 2. Testes com alguns modelos

#### 2.1 Textos usando para testes

In [36]:
textos = ['I do not like to study', 
          'I want to go to college', 
          'I will be a computer student',
          'I need money',
          'I like data science',
          'I used my emails years ago paying for a transaction',
          'Student, student, student.',
          'I am 24 years old',
          'I need to send an email to my professor']

#### 2.2 Usando o CountVectorizer

In [37]:
from sklearn.feature_extraction.text import CountVectorizer
#Pré-processamento de texto, tokenização e filtragem de palavras irrelevantes estão incluídos no CountVectorizer, 
#que cria um dicionário de recursos e transforma documentos em vetores de recursos.

In [38]:
count_vect = CountVectorizer(lowercase=True)
X_train = count_vect.fit_transform(X)

In [39]:
X_train.shape

(10000, 19980)

In [40]:
X_train[0]

<1x19980 sparse matrix of type '<class 'numpy.int64'>'
	with 20 stored elements in Compressed Sparse Row format>

In [41]:
X_train[0].todense()

matrix([[0, 0, 0, ..., 0, 0, 0]])

In [42]:
count_vect.vocabulary_.get(u'student')

16965

In [43]:
teste = count_vect.transform(textos)

In [44]:
from sklearn.naive_bayes import MultinomialNB
#Classificador para tentar prever a categoria de uma postagem.

In [45]:
nb = MultinomialNB()
nb.fit(X_train,y)

MultinomialNB()

In [46]:
preds = nb.predict(teste)

In [47]:
for doc, pred in zip(textos, preds):
    print(f'Texto: {doc} \nClasse: {pred}\n')

Texto: I do not like to study 
Classe: Credit reporting, credit repair services, or other personal consumer reports

Texto: I want to go to college 
Classe: Student loan

Texto: I will be a computer student 
Classe: Credit reporting, credit repair services, or other personal consumer reports

Texto: I need money 
Classe: Credit reporting, credit repair services, or other personal consumer reports

Texto: I like data science 
Classe: Credit reporting, credit repair services, or other personal consumer reports

Texto: I used my emails years ago paying for a transaction 
Classe: Credit card or prepaid card

Texto: Student, student, student. 
Classe: Student loan

Texto: I am 24 years old 
Classe: Credit reporting, credit repair services, or other personal consumer reports

Texto: I need to send an email to my professor 
Classe: Credit reporting, credit repair services, or other personal consumer reports



### 3. Separando a base em treino e teste

In [48]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y)

### 4. Avaliando o modelo CountVectorizer

In [49]:
X_test_countVect = count_vect.transform(X_test)

In [50]:
nb.score(X_test_countVect,y_test)

0.8224

### 5. Adicionando em pipelines

In [51]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier

#### 5.1 CountVectorizer com MultinomialNB

In [52]:
pipeline = Pipeline([
                        ('count_vect', CountVectorizer(lowercase=True,stop_words='english')),
                        ('clf', MultinomialNB())
                    ])

In [53]:
pipeline.fit(X_train, y_train)
pipeline.score(X_test, y_test)

0.7872

#### 5.2 CountVectorizer com RandomForestClassifier

In [54]:
pipeline = Pipeline([
                        ('count_vect', CountVectorizer(lowercase=True,stop_words='english')),
                        ('clf', RandomForestClassifier())
                    ])

In [55]:
pipeline.fit(X_train, y_train)
pipeline.score(X_test, y_test)

0.7572

#### 5.3 TfidfVectorizer com MultinomialNB

In [56]:
pipeline = Pipeline([
                        ('count_vect', TfidfVectorizer(lowercase=True,stop_words='english')),
                        ('clf', MultinomialNB())
                    ])


In [57]:
pipeline.fit(X_train,y_train)
pipeline.score(X_test,y_test)

0.5436

#### 5.3 CountVectorizer, TfidfTransformer e MultinomialNB

In [59]:
#TfidfTransformer - Transform a count matrix to a normalized tf or tf-idf representation
pipeline = Pipeline([
                         ('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()), #usando l1 deu um resultado pior que o l2
                         ('clf', MultinomialNB()),
                    ])

In [60]:
pipeline.fit(X_train,y_train)
pipeline.score(X_test,y_test)

0.51

#### 5.4 CountVectorizer, TfidfTransformer e SGDClassifier

In [61]:
pipeline = Pipeline([
                         ('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                               alpha=1e-3, random_state=42,
                                               max_iter=5, tol=None)),
                     ])

In [62]:
pipeline.fit(X_train,y_train)
pipeline.score(X_test,y_test)

0.7884